**Question 1) List two benefits of using Mongoose instead of the native MongoDB driver. Also mention one scenario where using the native driver could be preferable.**

**Answer:**

Schema & validation support – Mongoose lets you define schemas with built-in validation, defaults, and type casting, which helps maintain consistent data and reduces bugs.

Higher-level abstractions – It provides models, middleware (hooks), and population for handling relationships, making application code cleaner and easier to maintain.

**Question 2) Create a Mongoose schema for a `User` with properties:**  
- `name` (required string, minimum length 2, maximum length 50)  
- `email` (required string)  

Then export a `User` model.

**Answer:**

import mongoose from "mongoose";

const userSchema = new mongoose.Schema(
  {
    name: {
      type: String,
      required: true,
      minlength: 2,
      maxlength: 50,
      trim: true,
    },
    email: {
      type: String,
      required: true,
      trim: true,
      lowercase: true,
    },
  },
  { timestamps: true }
);

const User = mongoose.model("User", userSchema);

export default User;

**Question 3) Write code to (a) fetch all users, (b) fetch a user by ID, and (c) add a query helper byEmailDomain(domain) that returns users whose email ends with the given domain. Show example usage.**

**Answer:**

import mongoose from "mongoose";

/* =========================
   USER SCHEMA + MODEL
========================= */
const userSchema = new mongoose.Schema({
  name: String,
  email: String,
});

/* Query Helper */
userSchema.query.byEmailDomain = function (domain) {
  return this.where({
    email: { $regex: `@${domain}$`, $options: "i" },
  });
};

const User = mongoose.model("User", userSchema);

/* =========================
   (a) Fetch all users
========================= */
async function getAllUsers() {
  const users = await User.find();
  return users;
}

/* =========================
   (b) Fetch user by ID
========================= */
async function getUserById(userId) {
  const user = await User.findById(userId);
  return user;
}

/* =========================
   (c) Example usage of byEmailDomain
========================= */
async function getUsersByDomain() {
  const users = await User.find().byEmailDomain("gmail.com");
  return users;
}

/* =========================
   EXAMPLE CALLS
========================= */
(async () => {
  const allUsers = await getAllUsers();
  console.log("All Users:", allUsers);

  const singleUser = await getUserById("64f1a2b9c1234567890abcd");
  console.log("User by ID:", singleUser);

  const gmailUsers = await getUsersByDomain();
  console.log("Gmail Users:", gmailUsers);
})();

**Question 4) Demonstrate two ways to change a user’s name to "Rita" using Mongoose: (1) load, modify, then save(), and (2) use findOneAndUpdate(). Mention when save() is preferable.**

**Answer:**

import User from "./models/User.js";

/* =========================
   1) Load → Modify → Save
========================= */
async function updateWithSave(userId) {
  const user = await User.findById(userId); // load
  if (!user) return null;

  user.name = "Rita"; // modify
  await user.save(); // save

  return user;
}

/* =========================
   2) Using findOneAndUpdate
========================= */
async function updateWithFindOneAndUpdate(userId) {
  const updatedUser = await User.findOneAndUpdate(
    { _id: userId },
    { name: "Rita" },
    { new: true } // return updated document
  );

  return updatedUser;
}

**Question 5) Explain when you would embed a subdocument versus using references in MongoDB/Mongoose. Show code for a one-to-many reference (e.g. a Post having many Comment references).**

**Answer:**

Embed subdocuments when:

The related data is tightly coupled to the parent and rarely accessed on its own

The data is small in size and usually fetched together

You want atomic updates within a single document

import mongoose from "mongoose";

/* =========================
   COMMENT SCHEMA
========================= */
const commentSchema = new mongoose.Schema(
  {
    text: {
      type: String,
      required: true,
    },
    author: {
      type: mongoose.Schema.Types.ObjectId,
      ref: "User",
    },
  },
  { timestamps: true }
);

const Comment = mongoose.model("Comment", commentSchema);

/* =========================
   POST SCHEMA (references)
========================= */
const postSchema = new mongoose.Schema(
  {
    title: {
      type: String,
      required: true,
    },
    content: String,
    comments: [
      {
        type: mongoose.Schema.Types.ObjectId,
        ref: "Comment",
      },
    ],
  },
  { timestamps: true }
);

const Post = mongoose.model("Post", postSchema);

/* =========================
   Example: populate comments
========================= */
async function getPostWithComments(postId) {
  return await Post.findById(postId).populate("comments");
}

**Question 6) Write a Mongoose (or MongoDB) aggregation pipeline to group users by email domain and count how many users per domain. Then show how you could use $lookup to join a bounces collection (with bounce counts per domain).**

**Answer:**
// users collection: { name, email }

const pipeline = [
  {
    // extract domain from email
    $project: {
      domain: {
        $arrayElemAt: [{ $split: ["$email", "@"] }, 1],
      },
    },
  },
  {
    // group by domain
    $group: {
      _id: "$domain",
      userCount: { $sum: 1 },
    },
  },
  {
    // optional: nicer field names
    $project: {
      _id: 0,
      domain: "$_id",
      userCount: 1,
    },
  },
];

const usersPerDomain = await User.aggregate(pipeline);
console.log(usersPerDomain);
const pipelineWithLookup = [
  {
    $project: {
      domain: {
        $arrayElemAt: [{ $split: ["$email", "@"] }, 1],
      },
    },
  },
  {
    $group: {
      _id: "$domain",
      userCount: { $sum: 1 },
    },
  },
  {
    $lookup: {
      from: "bounces",        // collection name
      localField: "_id",      // domain from users
      foreignField: "domain", // domain in bounces
      as: "bounceInfo",
    },
  },
  {
    $unwind: {
      path: "$bounceInfo",
      preserveNullAndEmptyArrays: true, // keep domains with no bounces
    },
  },
  {
    $project: {
      _id: 0,
      domain: "$_id",
      userCount: 1,
      bounceCount: {
        $ifNull: ["$bounceInfo.bounceCount", 0],
      },
    },
  },
];

const result = await User.aggregate(pipelineWithLookup);
console.log(result);

**Question 7) Show minimal code (ESM) to connect to MongoDB Atlas using Mongoose. Assume the URI is stored in the environment variable MONGODB_URI.**

**Answer:**

import mongoose from "mongoose";

const connectDB = async () => {
  try {
    await mongoose.connect(process.env.MONGODB_URI);
    console.log("MongoDB connected");
  } catch (err) {
    console.error("MongoDB connection error:", err.message);
    process.exit(1);
  }
};

export default connectDB;

**Question 8) Give a short description of Express. Then write routes for:**
●	GET /health → responds “OK”
●	GET /users/:id → responds with { id: <id> }
●	GET /search?term=... → responds with { term: <term> }

**Answer:**
Express.js is a fast, minimal, and flexible Node.js web framework used to build APIs and web applications. It simplifies handling HTTP requests, routing, middleware, and responses.

import express from "express";

const app = express();

/* =========================
   Routes
========================= */

// GET /health → "OK"
app.get("/health", (req, res) => {
  res.send("OK");
});

// GET /users/:id → { id: <id> }
app.get("/users/:id", (req, res) => {
  const { id } = req.params;
  res.json({ id });
});

// GET /search?term=...
app.get("/search", (req, res) => {
  const { term } = req.query;
  res.json({ term });
});

/* =========================
   Start server
========================= */
const PORT = 3000;
app.listen(PORT, () => {
  console.log(`Server running on port ${PORT}`);
});

**Question 9) Create a router for /api/users with two endpoints: GET / returning empty array and POST / returning the posted JSON. Also add a logger middleware that prints method and URL for every request.**

**Answer:**
const logger = (req, res, next) => {
  console.log(`${req.method} ${req.originalUrl}`);
  next();
};
/api/users router (ESM)
import express from "express";

const router = express.Router();

/* =========================
   Routes
========================= */

// GET /api/users → []
router.get("/", (req, res) => {
  res.json([]);
});

// POST /api/users → echo posted JSON
router.post("/", (req, res) => {
  res.json(req.body);
});

export default router;
Use middleware + router in app
import express from "express";
import usersRouter from "./routes/users.js";

const app = express();

// middleware
app.use(express.json());
app.use(logger);

// routes
app.use("/api/users", usersRouter);

app.listen(3000, () => {
  console.log("Server running on port 3000");
});


**Question 10) Add an error-handling middleware to catch thrown errors and respond with { error: <message> }, status 500. Demonstrate by a route that throws an error.**

**Answer:**

import express from "express";

const app = express();

/* =========================
   Demo route that throws
========================= */
app.get("/boom", (req, res) => {
  throw new Error("Something went wrong!");
});

/* =========================
   Error-handling middleware
   (must have 4 params)
========================= */
app.use((err, req, res, next) => {
  console.error(err.message);
  res.status(500).json({
    error: err.message,
  });
});

/* =========================
   Start server
========================= */
app.listen(3000, () => {
  console.log("Server running on port 3000");
});

**Question 11) Write middleware auth() that checks for Authorization: Bearer <token> header, verifies the token, and sets req.user. Then protect route GET /me to return user info if valid, otherwise 401.**

**Answer:**

import jwt from "jsonwebtoken";

const auth = (req, res, next) => {
  const authHeader = req.headers.authorization;

  // Check header exists and format
  if (!authHeader || !authHeader.startsWith("Bearer ")) {
    return res.status(401).json({ error: "Unauthorized" });
  }

  const token = authHeader.split(" ")[1];

  try {
    // verify token (example using JWT)
    const decoded = jwt.verify(token, process.env.JWT_SECRET);

    // attach user info to request
    req.user = decoded;
    next();
  } catch (err) {
    return res.status(401).json({ error: "Invalid token" });
  }
};

export default auth;

import express from "express";
import auth from "./middleware/auth.js";

const app = express();

/* =========================
   Protected route
========================= */
app.get("/me", auth, (req, res) => {
  res.json({
    user: req.user,
  });
});

/* =========================
   Server
========================= */
app.listen(3000, () => {
  console.log("Server running on port 3000");
});






